# news_classification

In [1]:
import pandas as pd

## Load_dataset

In [2]:
df_train = pd.read_csv("./data/train_data.csv")
df_train.head()

,title,description,tags
0,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...,گروه استان‌ها- دانش‌آموزان مدرسه روستای کَهنان...,استانها
1,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...,گروه استان‌ها ــ انقلابی سرشناس بحرین با بیان ...,استانها
2,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...,گروه استان‌ها ــ با اعلام شیوع ویروس کرونا در ...,استانها
3,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...,یک متخصص بیوتکنولوژی پزشکی با بیان اینکه مرگ‌و...,اجتماعی
4,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...,رئیس اتحادیه فروشگاه‌های زنجیره‌ای با اشاره به...,اقتصادی


## EDA

In [3]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12457 entries, 0 to 12456
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        12457 non-null  object
 1   description  12457 non-null  object
 2   tags         12455 non-null  object
dtypes: object(3)
memory usage: 292.1+ KB


In [4]:
df_train.shape

(12457, 3)

### Missing value

In [5]:
df_train.isna().sum()

title          0
description    0
tags           2
dtype: int64

In [6]:
df_train[df_train["tags"].isna()]

,title,description,tags
11659,مجید آذرپی به زندان اوین بازگشت,مجید آذرپی، زندانی سیاسی اصلاح‌طلب و عضو ستاد ...,NaN
12186,مقام آموزش و پرورش: الزام دانش آموزان به پوشش ...,مدیرکل انجمن اولیا و مربیان وزارت آموزش و پرور...,NaN


In [7]:
df_train.dropna(inplace=True)

In [8]:
df_train[["description", "title"]].agg(
    lambda x: x.str.len()
    .agg(
        ["mean", "min", "max", "median", "count", "std"]
    )
).round(2)

,description,title
mean,2406.14,62.86
min,1.00,11.00
max,55609.00,187.00
median,1218.00,60.00
count,12455.00,12455.00
std,2952.85,19.42


In [9]:
df_train["description"].str.len().sort_values()[:15]

9851      1
10038     1
9756      3
9755      3
9752      3
322      43
1381     44
559      44
200      45
551      46
337      47
1587     49
2930     51
140      53
1316     54
Name: description, dtype: int64

In [10]:
df_train["description"][df_train["description"].str.len().isin([1, 3])]

9752     . \n
9755     . \n
9756     . \n
9851       \n
10038      \n
Name: description, dtype: object

In [11]:
invalid_description_index = df_train["description"][df_train["description"].str.len().isin([
    1, 3])].index
invalid_description_index

Index([9752, 9755, 9756, 9851, 10038], dtype='int64')

In [12]:
df_train.drop(invalid_description_index,inplace=True)

In [13]:
df_train.shape

(12450, 3)

### Create Text data form 'title' and 'description'

In [14]:
df_train["text"] = df_train["title"] + " " + df_train["description"]
df_train.drop(columns=["title","description"],inplace=True)
df_train

,tags,text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...
3,اجتماعی,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...
...,...,...
12452,سیاسی,علیرضا زاکانی رئیس مرکز پژوهش‌های مجلس شد به ن...
12453,سیاسی,قوه قضائیه: تقاضای فرجام در پرونده روح‌الله زم...
12454,بایگانی,روسیه: ابراز نگرانی لاوروف و ظریف درباره حضور ...
12455,بایگانی,وزیر خارجه ایران وارد بیروت شد در حالی که اعتر...


In [20]:
df_train["tags"].value_counts()

tags
جهان                  1657
ایران                 1188
بین الملل              972
سیاسی                  921
سیاسی > دولت           506
                      ... 
ورزشی > توپ و تور        1
بین الملل > فلسطین       1
استان ها > اردبیل        1
استان ها > یزد           1
استان ها > فارس          1
Name: count, Length: 99, dtype: int64

##### remove unnesesery column type 

In [19]:
df_train = df_train[df_train["tags"]!="بایگانی"]

In [21]:
df_train.head()

,tags,text
0,استانها,روایتی از تحصیل زیرسقف‌های لرزان و سرمای سوزان...
1,استانها,انقلابی سرشناس بحرین در گفت‌وگو با تسنیم: مردم...
2,استانها,‌زندگی مردم در شهر قم ‌جریان دارد / پیشگیری در...
3,اجتماعی,واکنش &quot;پروفسور کرمی&quot; به دروغ‌پراکنی ...
4,اقتصادی,مردم نگران تأمین کالاهای موردنیاز خود نباشند/ ...
